# Desafio Nasa 1

O objetivo deste notebook é responder as perguntas descritas a seguir:

1. Número de hosts únicos ?
2. O total de erros 404 ?
3. Os 5 URLs que mais causaram erro 404 ?
4. Quantidade de erros 404 por dia ?
5. O total de bytes retornados ?


Para obtenção das respostas, serão usados os seguintes datasets

 - (https://github.com/paaarx/semantix/blob/master/NASA_access_log_Aug95.gz)
 - (https://github.com/paaarx/semantix/blob/master/NASA_access_log_Jul95.gz)

In [0]:
%sh

pwd

/databricks/driver


In [0]:
%sh ls -l

total 816
drwxr-xr-x 2 root root   4096 Nov 10 11:56 conf
drwxr-xr-x 3 root root   4096 Nov 10 11:58 eventlogs
drwxr-xr-x 2 root root   4096 Nov 10 12:15 ganglia
drwxr-xr-x 2 root root   4096 Nov 10 12:03 logs
drwxr-xr-x 5 root root   4096 Nov 10 12:03 metastore_db
-r-xr-xr-x 1 root root 813069 Nov 10 11:56 preload_class.lst


## Obtendo conjunto de dados - Extract

In [0]:
%sh wget https://github.com/paaarx/semantix/raw/master/NASA_access_log_Aug95.gz

--2021-11-10 12:26:46--  https://github.com/paaarx/semantix/raw/master/NASA_access_log_Aug95.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Aug95.gz [following]
--2021-11-10 12:26:46--  https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Aug95.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16633316 (16M) [application/octet-stream]
Saving to: ‘NASA_access_log_Aug95.gz’

     0K .......... .......... .......... .......... ..........  0% 7.07M 2s
    50K .......... .......... .......... .......... ..........  0% 6.92M 2s
   100

In [0]:
%sh wget https://github.com/paaarx/semantix/raw/master/NASA_access_log_Jul95.gz

--2021-11-10 12:26:49--  https://github.com/paaarx/semantix/raw/master/NASA_access_log_Jul95.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Jul95.gz [following]
--2021-11-10 12:26:49--  https://raw.githubusercontent.com/paaarx/semantix/master/NASA_access_log_Jul95.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20676672 (20M) [application/octet-stream]
Saving to: ‘NASA_access_log_Jul95.gz’

     0K .......... .......... .......... .......... ..........  0% 5.26M 4s
    50K .......... .......... .......... .......... ..........  0% 5.63M 4s
   100

In [0]:
%sh ls -lh

total 37M
-rw-r--r-- 1 root root  16M Nov 10 12:26 NASA_access_log_Aug95.gz
-rw-r--r-- 1 root root  20M Nov 10 12:26 NASA_access_log_Jul95.gz
drwxr-xr-x 2 root root 4.0K Nov 10 11:56 conf
drwxr-xr-x 3 root root 4.0K Nov 10 11:58 eventlogs
drwxr-xr-x 2 root root 4.0K Nov 10 12:15 ganglia
drwxr-xr-x 2 root root 4.0K Nov 10 12:03 logs
drwxr-xr-x 5 root root 4.0K Nov 10 12:03 metastore_db
-r-xr-xr-x 1 root root 795K Nov 10 11:56 preload_class.lst


In [0]:
! gzip -lv ./NASA_access_log_Aug95.gz

method  crc     date  time           compressed        uncompressed  ratio uncompressed_name
defla 89529258 Nov 10 12:26            16633316           167813770  90.1% ./NASA_access_log_Aug95


In [0]:
! gzip -lv ./NASA_access_log_Jul95.gz

method  crc     date  time           compressed        uncompressed  ratio uncompressed_name
defla 6656c0d8 Nov 10 12:26            20676672           205242368  89.9% ./NASA_access_log_Jul95


In [0]:
%sh gzip -dk ./NASA_access_log_Aug95.gz

In [0]:
%sh gzip -dk ./NASA_access_log_Jul95.gz

In [0]:
%sh ls -lh

total 393M
-rw-r--r-- 1 root root 161M Nov 10 12:26 NASA_access_log_Aug95
-rw-r--r-- 1 root root  16M Nov 10 12:26 NASA_access_log_Aug95.gz
-rw-r--r-- 1 root root 196M Nov 10 12:26 NASA_access_log_Jul95
-rw-r--r-- 1 root root  20M Nov 10 12:26 NASA_access_log_Jul95.gz
drwxr-xr-x 2 root root 4.0K Nov 10 11:56 conf
drwxr-xr-x 3 root root 4.0K Nov 10 11:58 eventlogs
drwxr-xr-x 2 root root 4.0K Nov 10 12:15 ganglia
drwxr-xr-x 2 root root 4.0K Nov 10 12:03 logs
drwxr-xr-x 5 root root 4.0K Nov 10 12:03 metastore_db
-r-xr-xr-x 1 root root 795K Nov 10 11:56 preload_class.lst


In [0]:
! head -n 2 NASA_access_log_Aug95

in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839
uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0


In [0]:
! head -n 2 NASA_access_log_Jul95

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985


In [0]:
%sh mkdir Nasa_data

In [0]:
%sh

mv ./NASA_access_log_Jul95 ./Nasa_data/NASA_access_log_0Jul95
mv ./NASA_access_log_Aug95 ./Nasa_data/NASA_access_log_1Aug95

In [0]:
! grep -rnw '/databricks/driver/Nasa_data/' -e ','

/databricks/driver/Nasa_data/NASA_access_log_1Aug95:712000:dsl.rhilinet.gov - - [16/Aug/1995:11:17:54 -0400] "GET /software/winvn/winvn.html>WinVN</a>, a" 404 -


In [0]:
! grep -rnw '/databricks/driver/Nasa_data/' -e '|'

Binary file /databricks/driver/Nasa_data/NASA_access_log_0Jul95 matches
Binary file /databricks/driver/Nasa_data/NASA_access_log_1Aug95 matches


Importando todos os dados para uma única coluna.

In [0]:
df_raw1 = (
  spark.read
  .option("sep", "|")
  .option("header", False)
  .schema("line String")
  .csv("file:/databricks/driver/Nasa_data")
)

display(df_raw1)

line
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245"
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985"
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085"
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0"
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179"
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0"
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0"
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985"
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985"
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074"


**Observação**

Uma possível melhor prática talvez seja criar um programa em Python ou *Shell Script* para checar as informações dos arquivos de log e selecionar apenas as informações de interesse.
Contudo, serão usados os recursos do Pyspark para aplicar os tratamentos necessários aos dados.

## Tratamento dos dados - Transform

In [0]:
df_raw1.count()

Out[18]: 3461613

In [0]:
from pyspark.sql.functions import split

df_t1 = df_raw1.withColumn("origin", split(df_raw1["line"], " - - ").getItem(0) )

display(df_t1)

line,origin
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",199.72.81.55
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",unicomp6.unicomp.net
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",199.120.110.21
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",burger.letters.com
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",199.120.110.21
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",burger.letters.com
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",burger.letters.com
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",205.212.115.106
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",d104.aa.net
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",129.94.144.152


In [0]:
df_t1.filter(df_t1['origin'].rlike("[A-z0-9\.]+")).count() 

Out[20]: 3461613

In [0]:
from pyspark.sql.functions import regexp_extract

df_t2 = (df_t1.withColumn("date_time", regexp_extract(df_t1['line'], '([0-9]+\\/(Jul|Aug)\\/[0-9]+:[0-9]+:[0-9]+:[0-9]+\\s\\-[0-9]+)', 1)))

display(df_t2)

line,origin,date_time
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",199.72.81.55,01/Jul/1995:00:00:01 -0400
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",unicomp6.unicomp.net,01/Jul/1995:00:00:06 -0400
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",199.120.110.21,01/Jul/1995:00:00:09 -0400
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",burger.letters.com,01/Jul/1995:00:00:11 -0400
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",199.120.110.21,01/Jul/1995:00:00:11 -0400
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",burger.letters.com,01/Jul/1995:00:00:12 -0400
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",burger.letters.com,01/Jul/1995:00:00:12 -0400
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",205.212.115.106,01/Jul/1995:00:00:12 -0400
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",d104.aa.net,01/Jul/1995:00:00:13 -0400
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",129.94.144.152,01/Jul/1995:00:00:13 -0400


In [0]:
df_t2.filter(~df_t2['date_time'].rlike("([0-9]+\\/(Jul|Aug)\\/[0-9]+:[0-9]+:[0-9]+:[0-9]+\\s\\-[0-9]+)")).count()

Out[23]: 1

In [0]:
display(df_t2.filter(~df_t2['date_time'].rlike("([0-9]+\\/(Jul|Aug)\\/[0-9]+:[0-9]+:[0-9]+:[0-9]+\\s\\-[0-9]+)")) )

line,origin,date_time
alyssa.p,alyssa.p,


In [0]:
df_t3 = (
  df_t2.withColumn("http_command", regexp_extract(df_t2['line'], '\\"(GET|POST|HEAD|PUT|DELETE|OPTIONS|TRACE|PATCH)', 1))
)

display(df_t3.select("line", "http_command"))

line,http_command
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",GET
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",GET
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",GET
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",GET
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",GET
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",GET
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",GET
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",GET
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",GET
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",GET


In [0]:
df_t3.filter(~df_t3['http_command'].rlike("(GET|POST|HEAD|PUT|DELETE|OPTIONS|TRACE|PATCH)")).count()

Out[27]: 16

In [0]:
display(
  df_t3.filter(~df_t3['http_command'].rlike("(GET|POST|HEAD|PUT|DELETE|OPTIONS|TRACE|PATCH)")).select("line", "http_command")
)

line,http_command
"klothos.crl.research.digital.com - - [10/Jul/1995:16:45:50 -0400] """" 400 -",
"firewall.dfw.ibm.com - - [20/Jul/1995:07:34:34 -0400] ""1/history/apollo/images/"" 400 -",
"firewall.dfw.ibm.com - - [20/Jul/1995:07:53:24 -0400] ""1/history/apollo/images/"" 400 -",
"128.159.122.20 - - [20/Jul/1995:15:28:50 -0400] ""k��tx��tG��t̓�"" 400 -",
"128.159.122.20 - - [24/Jul/1995:13:52:50 -0400] ""k��tx��tG��t̓�"" 400 -",
"128.159.144.83 - - [14/Aug/1995:15:51:37 -0400] ""��6��T7�F��F"" 400 -",
"128.159.144.83 - - [14/Aug/1995:15:51:58 -0400] ""��6��T7�F��F"" 400 -",
"128.159.144.83 - - [14/Aug/1995:15:52:23 -0400] ""��6��T7�F��F"" 400 -",
"128.159.144.83 - - [14/Aug/1995:15:59:35 -0400] ""��6��T7�F��F"" 400 -",
"bloor.torfree.net - - [22/Aug/1995:14:55:16 -0400] ""huttle/countdown/"" 400 -",


In [0]:
df_t4 = (
  df_t3.withColumn("url", regexp_extract(df_t3['line'], '\\"(GET|POST|HEAD|PUT|DELETE|OPTIONS|TRACE|PATCH) (.*)( HTTP(.*)|\\")', 2))
)

display(df_t4.select("line", "url"))

line,url
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",/history/apollo/ HTTP/1.0
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",/shuttle/countdown/ HTTP/1.0
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",/shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",/shuttle/countdown/liftoff.html HTTP/1.0
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",/shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",/images/NASA-logosmall.gif HTTP/1.0
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",/shuttle/countdown/video/livevideo.gif HTTP/1.0
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",/shuttle/countdown/countdown.html HTTP/1.0
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",/shuttle/countdown/ HTTP/1.0
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",/ HTTP/1.0


In [0]:
from pyspark.sql.functions import regexp_replace

df_t5 = (
  df_t4.withColumn("url", regexp_replace(df_t4['url'], ' HTTP(.*)', "") )
)
  
display(df_t5.select("line", "url"))

line,url
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",/history/apollo/
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",/shuttle/countdown/
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",/shuttle/missions/sts-73/mission-sts-73.html
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",/shuttle/countdown/liftoff.html
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",/shuttle/missions/sts-73/sts-73-patch-small.gif
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",/images/NASA-logosmall.gif
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",/shuttle/countdown/video/livevideo.gif
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",/shuttle/countdown/countdown.html
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",/shuttle/countdown/
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",/


In [0]:
display(
  df_t5.filter(df_t5["url"] == "").select("line", "url")
)

line,url
"async13.spherenet.com - - [02/Jul/1995:10:01:16 -0400] ""GET /shuttle/missions/sts-71/images/file:C",
"async13.spherenet.com - - [02/Jul/1995:10:01:45 -0400] ""GET /shuttle/missions/sts-71/images/file:C",
"klothos.crl.research.digital.com - - [10/Jul/1995:16:45:50 -0400] """" 400 -",
"jumbo.jet.uk - - [11/Jul/1995:08:06:29 -0400] ""GET HTTP/1.0"" 302 -",
"202.251.172.60 - - [14/Jul/1995:04:10:23 -0400] ""GET /shuttle/missions/sts-65/mission-sts-65.htmlh����������",
"firewall.dfw.ibm.com - - [20/Jul/1995:07:34:34 -0400] ""1/history/apollo/images/"" 400 -",
"firewall.dfw.ibm.com - - [20/Jul/1995:07:53:24 -0400] ""1/history/apollo/images/"" 400 -",
"128.159.122.20 - - [20/Jul/1995:15:28:50 -0400] ""k��tx��tG��t̓�"" 400 -",
"128.159.122.20 - - [24/Jul/1995:13:52:50 -0400] ""k��tx��tG��t̓�"" 400 -",
"203.16.174.5 - - [14/Aug/1995:04:12:17 -0400] ""GET HTTP/1.0"" 302 -",


In [0]:
from pyspark.sql.functions import when

df_t6 = (  
    df_t5.withColumn("url",
      when(
          df_t5["url"] == ""
          , regexp_extract(df_t5["line"], '\\"(.*)(\\"|:C)', 1) 
      ).otherwise(df_t5["url"]) 
    ) 
)

In [0]:
df_t7 = (
  df_t6.withColumn("url", 
    when( df_t6["url"] == "GET  HTTP/1.0", "").otherwise(df_t6["url"]) 
  )
)

display(df_t7)

line,origin,date_time,http_command,url
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",199.72.81.55,01/Jul/1995:00:00:01 -0400,GET,/history/apollo/
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",unicomp6.unicomp.net,01/Jul/1995:00:00:06 -0400,GET,/shuttle/countdown/
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",199.120.110.21,01/Jul/1995:00:00:09 -0400,GET,/shuttle/missions/sts-73/mission-sts-73.html
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",burger.letters.com,01/Jul/1995:00:00:11 -0400,GET,/shuttle/countdown/liftoff.html
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",199.120.110.21,01/Jul/1995:00:00:11 -0400,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",burger.letters.com,01/Jul/1995:00:00:12 -0400,GET,/images/NASA-logosmall.gif
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",burger.letters.com,01/Jul/1995:00:00:12 -0400,GET,/shuttle/countdown/video/livevideo.gif
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",205.212.115.106,01/Jul/1995:00:00:12 -0400,GET,/shuttle/countdown/countdown.html
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",d104.aa.net,01/Jul/1995:00:00:13 -0400,GET,/shuttle/countdown/
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",129.94.144.152,01/Jul/1995:00:00:13 -0400,GET,/


In [0]:
display(df_t7.filter(df_t7["url"] == "").select("line", "url"))

line,url
"jumbo.jet.uk - - [11/Jul/1995:08:06:29 -0400] ""GET HTTP/1.0"" 302 -",
"202.251.172.60 - - [14/Jul/1995:04:10:23 -0400] ""GET /shuttle/missions/sts-65/mission-sts-65.htmlh����������",
"203.16.174.5 - - [14/Aug/1995:04:12:17 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:20 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:21 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:22 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:27 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:28 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:29 -0400] ""GET HTTP/1.0"" 302 -",
"mac391s.ksc.nasa.gov - - [23/Aug/1995:10:35:30 -0400] ""GET HTTP/1.0"" 302 -",


In [0]:
df_t8 = (
  df_t7.withColumn("status_http", regexp_extract(df_t7["line"], '(\\d*) ((\\d*)|\\-)$', 1)  )
  .withColumn("bytes", regexp_extract(df_t7["line"], '(\\d*) ((\\d*)|\\-)$', 2)  )
)

display(df_t8.select("line", "status_http", "bytes"))

line,status_http,bytes
"199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ""GET /history/apollo/ HTTP/1.0"" 200 6245",200,6245
"unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",200,3985
"199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] ""GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0"" 200 4085",200,4085
"burger.letters.com - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/countdown/liftoff.html HTTP/1.0"" 304 0",304,0
"199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] ""GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0"" 200 4179",200,4179
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /images/NASA-logosmall.gif HTTP/1.0"" 304 0",304,0
"burger.letters.com - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/video/livevideo.gif HTTP/1.0"" 200 0",200,0
"205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] ""GET /shuttle/countdown/countdown.html HTTP/1.0"" 200 3985",200,3985
"d104.aa.net - - [01/Jul/1995:00:00:13 -0400] ""GET /shuttle/countdown/ HTTP/1.0"" 200 3985",200,3985
"129.94.144.152 - - [01/Jul/1995:00:00:13 -0400] ""GET / HTTP/1.0"" 200 7074",200,7074


In [0]:
display(
  df_t8.filter(df_t8["status_http"] == "").select("line", "status_http", "bytes")
)

line,status_http,bytes
"async13.spherenet.com - - [02/Jul/1995:10:01:16 -0400] ""GET /shuttle/missions/sts-71/images/file:C",,
"async13.spherenet.com - - [02/Jul/1995:10:01:45 -0400] ""GET /shuttle/missions/sts-71/images/file:C",,
"202.251.172.60 - - [14/Jul/1995:04:10:23 -0400] ""GET /shuttle/missions/sts-65/mission-sts-65.htmlh����������",,
"163.206.42.13 - - [31/Aug/1995:11:04:42 -0400] ""�",,
"163.206.42.13 - - [31/Aug/1995:11:04:49 -0400] ""�",,
alyssa.p,,


In [0]:
df_t8.filter(df_t8["bytes"] == "").count()

Out[39]: 6

In [0]:
display(df_t8.filter(df_t8["bytes"] == "-").select("line", "bytes"))

line,bytes
"dd15-062.compuserve.com - - [01/Jul/1995:00:01:12 -0400] ""GET /news/sci.space.shuttle/archive/sci-space-shuttle-22-apr-1995-40.txt HTTP/1.0"" 404 -",-
"dynip42.efn.org - - [01/Jul/1995:00:02:14 -0400] ""GET /software HTTP/1.0"" 302 -",-
"ix-or10-06.ix.netcom.com - - [01/Jul/1995:00:02:40 -0400] ""GET /software/winvn HTTP/1.0"" 302 -",-
"ix-or10-06.ix.netcom.com - - [01/Jul/1995:00:03:24 -0400] ""GET /software HTTP/1.0"" 302 -",-
"link097.txdirect.net - - [01/Jul/1995:00:05:06 -0400] ""GET /shuttle HTTP/1.0"" 302 -",-
"ix-war-mi1-20.ix.netcom.com - - [01/Jul/1995:00:05:13 -0400] ""GET /shuttle/missions/sts-78/news HTTP/1.0"" 302 -",-
"ix-war-mi1-20.ix.netcom.com - - [01/Jul/1995:00:05:58 -0400] ""GET /shuttle/missions/sts-72/news HTTP/1.0"" 302 -",-
"netport-27.iu.net - - [01/Jul/1995:00:10:19 -0400] ""GET /pub/winvn/readme.txt HTTP/1.0"" 404 -",-
"netport-27.iu.net - - [01/Jul/1995:00:10:28 -0400] ""GET /pub/winvn/readme.txt HTTP/1.0"" 404 -",-
"dynip38.efn.org - - [01/Jul/1995:00:10:50 -0400] ""GET /software HTTP/1.0"" 302 -",-


In [0]:
df_t9 = (
  df_t8.withColumn("bytes", 
   when(df_t8["bytes"] == "-", "0")
   .when(df_t8["bytes"] == "", "0")
   .otherwise(df_t8["bytes"])
  )
)

display(df_t9.filter(df_t9.status_http == "404").select("line", "bytes"))

line,bytes
"dd15-062.compuserve.com - - [01/Jul/1995:00:01:12 -0400] ""GET /news/sci.space.shuttle/archive/sci-space-shuttle-22-apr-1995-40.txt HTTP/1.0"" 404 -",0
"netport-27.iu.net - - [01/Jul/1995:00:10:19 -0400] ""GET /pub/winvn/readme.txt HTTP/1.0"" 404 -",0
"netport-27.iu.net - - [01/Jul/1995:00:10:28 -0400] ""GET /pub/winvn/readme.txt HTTP/1.0"" 404 -",0
"blv-pm0-ip28.halcyon.com - - [01/Jul/1995:00:14:17 -0400] ""GET /persons/astronauts/i-to-l/lousmaJR.txt HTTP/1.0"" 404 -",0
"blv-pm0-ip28.halcyon.com - - [01/Jul/1995:00:14:32 -0400] ""GET /persons/astronauts/a-to-d/beanAL.txt HTTP/1.0"" 404 -",0
"cu-dialup-1005.cit.cornell.edu - - [01/Jul/1995:00:18:39 -0400] ""GET /pub/winvn/readme.txt HTTP/1.0"" 404 -",0
"cu-dialup-1005.cit.cornell.edu - - [01/Jul/1995:00:18:45 -0400] ""GET /pub/winvn/readme.txt HTTP/1.0"" 404 -",0
"cu-dialup-1005.cit.cornell.edu - - [01/Jul/1995:00:18:49 -0400] ""GET /pub/winvn/release.txt HTTP/1.0"" 404 -",0
"mimas.execpc.com - - [01/Jul/1995:00:18:59 -0400] ""GET /shuttle/missions/technology/sts-newsref/stsref-toc.html HTTP/1.0"" 404 -",0
"zoom112.telepath.com - - [01/Jul/1995:00:25:27 -0400] ""GET /history/apollo-13/apollo-13.html HTTP/1.0"" 404 -",0


In [0]:
display(df_t9.filter(df_t9.status_http == "").select("line", "bytes"))

line,bytes
"async13.spherenet.com - - [02/Jul/1995:10:01:16 -0400] ""GET /shuttle/missions/sts-71/images/file:C",0
"async13.spherenet.com - - [02/Jul/1995:10:01:45 -0400] ""GET /shuttle/missions/sts-71/images/file:C",0
"202.251.172.60 - - [14/Jul/1995:04:10:23 -0400] ""GET /shuttle/missions/sts-65/mission-sts-65.htmlh����������",0
"163.206.42.13 - - [31/Aug/1995:11:04:42 -0400] ""�",0
"163.206.42.13 - - [31/Aug/1995:11:04:49 -0400] ""�",0
alyssa.p,0


In [0]:
display(df_t9.filter(df_t9.status_http == "").select("line", "url"))

line,url
"async13.spherenet.com - - [02/Jul/1995:10:01:16 -0400] ""GET /shuttle/missions/sts-71/images/file:C",GET /shuttle/missions/sts-71/images/file
"async13.spherenet.com - - [02/Jul/1995:10:01:45 -0400] ""GET /shuttle/missions/sts-71/images/file:C",GET /shuttle/missions/sts-71/images/file
"202.251.172.60 - - [14/Jul/1995:04:10:23 -0400] ""GET /shuttle/missions/sts-65/mission-sts-65.htmlh����������",
"163.206.42.13 - - [31/Aug/1995:11:04:42 -0400] ""�",
"163.206.42.13 - - [31/Aug/1995:11:04:49 -0400] ""�",
alyssa.p,


In [0]:
df_t10 = (
  df_t9.withColumn("url",
    when(df_t9.status_http == "", regexp_extract(df_t9["line"], '\\"GET (.*)', 1) )
    .otherwise(df_t9.url)
  )
)

display(df_t10.filter(df_t10.status_http == "").select("line", "url"))

line,url
"async13.spherenet.com - - [02/Jul/1995:10:01:16 -0400] ""GET /shuttle/missions/sts-71/images/file:C",/shuttle/missions/sts-71/images/file:C
"async13.spherenet.com - - [02/Jul/1995:10:01:45 -0400] ""GET /shuttle/missions/sts-71/images/file:C",/shuttle/missions/sts-71/images/file:C
"202.251.172.60 - - [14/Jul/1995:04:10:23 -0400] ""GET /shuttle/missions/sts-65/mission-sts-65.htmlh����������",/shuttle/missions/sts-65/mission-sts-65.htmlh����������
"163.206.42.13 - - [31/Aug/1995:11:04:42 -0400] ""�",
"163.206.42.13 - - [31/Aug/1995:11:04:49 -0400] ""�",
alyssa.p,


In [0]:
df_t10.printSchema()

root
 |-- line: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- http_command: string (nullable = true)
 |-- url: string (nullable = true)
 |-- status_http: string (nullable = true)
 |-- bytes: string (nullable = true)



Para responder as perguntas, a coluna "bytes" precisa ser convertida para o tipo inteiro. Não há esta necessidade para a coluna "status_http".  
Considerando o dataset usado, não existe a necessidade de conversão de tipo da coluna "date_time".  
A coluna origina, "line", não é , mais necessária.

In [0]:
df_t11 = (
  df_t10.withColumn("bytes", df_t10["bytes"].cast("Int") )
  .select("origin", "date_time", "http_command", "url", "status_http", "bytes")
)

df_t11.printSchema()

root
 |-- origin: string (nullable = true)
 |-- date_time: string (nullable = true)
 |-- http_command: string (nullable = true)
 |-- url: string (nullable = true)
 |-- status_http: string (nullable = true)
 |-- bytes: integer (nullable = true)



## Respostas

### 1. Número de hosts únicos ?

In [0]:
df_t11.select("origin").distinct().count()

Out[59]: 137979

### 2. O total de erros 404 ?

In [0]:
df_t11.filter(df_t11["status_http"] == "404").count()

Out[60]: 20898

### 3. Os 5 URLs que mais causaram erro 404 ?

In [0]:
df_r3 = df_t11.filter(df_t11["status_http"] == "404").groupBy("url").count()
df_r3 = df_r3.withColumnRenamed("count", "total")
df_r3 = df_r3.sort(df_r3["total"].desc())

display(df_r3.limit(10))

url,total
/pub/winvn/readme.txt,2004
/pub/winvn/release.txt,1732
/shuttle/missions/STS-69/mission-STS-69.html,683
/shuttle/missions/sts-68/ksc-upclose.gif,428
/history/apollo/a-001/a-001-patch-small.gif,384
/history/apollo/sa-1/sa-1-patch-small.gif,383
/://spacelink.msfc.nasa.gov,381
/images/crawlerway-logo.gif,374
/elv/DELTA/uncons.htm,372
/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif,359


### 4. Quantidade de erros 404 por dia ?

In [0]:
from pyspark.sql.functions import concat, lit

df_r4 = (
  df_t11.filter(df_t11["status_http"] == "404")
  .withColumn("month_day", 
    concat(regexp_extract(df_t11["date_time"], "(Aug|Jul)", 1), lit("-"), regexp_extract(df_t11["date_time"], "(\\d*)", 1)  ) 
   )
)


display(df_r4.groupBy("month_day").count())

month_day,count
Jul-01,316
Jul-03,474
Jul-06,640
Jul-04,359
Jul-05,497
Jul-02,289
Jul-09,348
Jul-12,471
Jul-07,570
Jul-10,398


Resposta ordenanda por data

In [0]:
from pyspark.sql.functions import concat, lit, substring, col, count

df_r4 = (
  df_t11.filter(df_t11["status_http"] == "404")
  .withColumn("month_day", 
    concat(regexp_extract(df_t11["date_time"], "(Aug|Jul)", 1), lit("-"), regexp_extract(df_t11["date_time"], "(\\d*)", 1)  ) 
   )
  .withColumn("month_day_sort", 
    when(substring(col("month_day"), 1, 3)  == "Jul", concat(lit("7"), substring(col("month_day"), 5, 2)  ) )
    .when(substring(col("month_day"), 1, 3)  == "Aug", concat(lit("8"), substring(col("month_day"), 5, 2)  ) )
   )
)

display(df_r4.groupBy("month_day_sort").agg(
    count("month_day_sort").alias("total")
  ).sort(col("month_day_sort"))
  .withColumn("month_day", 
     when(substring(col("month_day_sort"), 1, 1) == "7", concat(lit("Jul-"), substring(col("month_day_sort"), 2, 2)) )
     .when(substring(col("month_day_sort"), 1, 1) == "8", concat(lit("Aug-"), substring(col("month_day_sort"), 2, 2)) )
  ).select("month_day", "total")
)

month_day,total
Jul-01,316
Jul-02,289
Jul-03,474
Jul-04,359
Jul-05,497
Jul-06,640
Jul-07,570
Jul-08,302
Jul-09,348
Jul-10,398


### 5. O total de bytes retornados ?

In [0]:
from pyspark.sql.functions import sum

display(df_t11.select(sum(df_t11["bytes"]).alias("Returned_Bytes")))

Returned_Bytes
65524314915


In [0]:
int_returned_bytes = df_t11.select(sum(df_t11["bytes"]) ).collect()[0]["sum(bytes)"]

print("Bytes retornados: ", int_returned_bytes)
print("KiloBytes retornados: ", int_returned_bytes / 1024)
print("MegaBytes retornados: ", int_returned_bytes / (1024) ** 2)
print("GigaBytes retornados: ", int_returned_bytes / (1024) ** 3)

Bytes retornados:  65524314915
KiloBytes retornados:  63988588.78417969
MegaBytes retornados:  62488.856234550476
GigaBytes retornados:  61.0242736665532
